In [1]:
# nltk module
import nltk
from nltk.stem.lancaster import LancasterStemmer

# module we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random
import json


Using TensorFlow backend.


In [2]:
# create a stemmer
stemmer = LancasterStemmer()

# load intents
# import our chat-bot intents file
import json
with open('intents_robi_en.json') as json_data:
    intents = json.load(json_data)




In [3]:
words = []
classes = []
documents = []
ignore_words = []


for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word into a sentence
        w = nltk.word_tokenize(pattern)
        # add word to our words list
        words.extend(w)
        # add documents to our corpus
        documents.append((w, intent['tag']))

        # add to our classses
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


# stem and lower case the word and remove duplicates
words =  [stemmer.stem(w.lower()) for w in words ]




# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))

In [4]:

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

51 documents
8 classes ['Robi_apps', 'balance', 'customer_care', 'goodbye', 'greeting', 'robi', 'social_media_pack', 'thanks']
74 unique stemmed words ["'s", ',', '?', 'account', 'any', 'anyon', 'ap', 'ar', 'awesom', 'bal', 'bil', 'by', 'bye', 'can', 'car', 'chat', 'check', 'cod', 'custom', 'day', 'do', 'download', 'facil', 'fb', 'for', 'get', 'good', 'goodby', 'hello', 'help', 'helplin', 'hi', 'hlw', 'how', 'i', 'imo', 'info', 'internet', 'is', 'lat', 'link', 'loc', 'me', 'med', 'minut', 'my', 'near', 'next', 'nic', 'no', 'numb', 'of', 'off', 'or', 'pack', 'peopl', 'postpaid', 'privileg', 'rob', 'see', 'serv', 'soc', 'spec', 'thank', 'that', 'the', 'ther', 'til', 'tim', 'to', 'what', 'whatsap', 'wher', 'you']


In [5]:
# # create our training data
# training = []
# # create an empty array for our output
# output_empty = [0] * len(classes)
# for doc in documents:
#     bag = []
#     pattern_words = doc[0]
#     pattern_words = [stemmer.stem(word.lower()) for word in pattern_words ]
    
#     for w in words:
#         bag.append(1) if w in pattern_words else bag.append(0)
        
#     output_row = list(output_empty)
#     output_row[classes.index(doc[1])] = 1
#     traininga.append(bag, output_row)


# output_row

In [6]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

In [7]:
training[0]

array([list([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 1, 0])], dtype=object)

In [8]:

# create train and test lists. X - patterns, Y - intents

train_x = list(training[:,0])

train_y = list(training[:,1])

print(train_x[:1], train_y[:1])
len(train_x), len(train_y)

[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] [[0, 0, 0, 0, 0, 0, 1, 0]]


(51, 51)

In [9]:

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons

# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [10]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               9600      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 520       
Total params: 18,376
Trainable params: 18,376
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Fit the model
history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
51/51 [==============================] - 0s 2ms/step - loss: 2.1119 - accuracy: 0.2157
Epoch 2/200
51/51 [==============================] - 0s 346us/step - loss: 2.0016 - accuracy: 0.2549
Epoch 3/200
51/51 [==============================] - 0s 364us/step - loss: 1.9337 - accuracy: 0.2549
Epoch 4/200
51/51 [==============================] - 0s 248us/step - loss: 1.7859 - accuracy: 0.3529
Epoch 5/200
51/51 [==============================] - 0s 445us/step - loss: 1.6452 - accuracy: 0.4902
Epoch 6/200
51/51 [==============================] - 0s 328us/step - loss: 1.6531 - accuracy: 0.4118
Epoch 7/200
51/51 [==============================] - 0s 367us/step - loss: 1.3728 - accuracy: 0.6078
Epoch 8/200
51/51 [==============================] - 0s 323us/step - loss: 1.3563 - accuracy: 0.6078
Epoch 9/200
51/51 [==============================] - 0s 266us/step - loss: 1.2778 - accuracy: 0.5686
Epoch 10/200
51/51 [==============================] - 0s 316us/step - loss: 1.1198 - accuracy

51/51 [==============================] - 0s 389us/step - loss: 0.0470 - accuracy: 1.0000
Epoch 82/200
51/51 [==============================] - 0s 442us/step - loss: 0.0344 - accuracy: 1.0000
Epoch 83/200
51/51 [==============================] - 0s 357us/step - loss: 0.0148 - accuracy: 1.0000
Epoch 84/200
51/51 [==============================] - 0s 360us/step - loss: 0.0277 - accuracy: 1.0000
Epoch 85/200
51/51 [==============================] - 0s 352us/step - loss: 0.0618 - accuracy: 0.9608
Epoch 86/200
51/51 [==============================] - 0s 464us/step - loss: 0.0146 - accuracy: 1.0000
Epoch 87/200
51/51 [==============================] - 0s 359us/step - loss: 0.0182 - accuracy: 1.0000
Epoch 88/200
51/51 [==============================] - 0s 344us/step - loss: 0.0558 - accuracy: 0.9804
Epoch 89/200
51/51 [==============================] - 0s 437us/step - loss: 0.0145 - accuracy: 1.0000
Epoch 90/200
51/51 [==============================] - 0s 361us/step - loss: 0.0271 - accuracy: 

51/51 [==============================] - 0s 352us/step - loss: 0.0511 - accuracy: 0.9804
Epoch 161/200
51/51 [==============================] - 0s 354us/step - loss: 0.0254 - accuracy: 0.9804
Epoch 162/200
51/51 [==============================] - 0s 422us/step - loss: 0.0267 - accuracy: 1.0000
Epoch 163/200
51/51 [==============================] - 0s 364us/step - loss: 0.0232 - accuracy: 1.0000
Epoch 164/200
51/51 [==============================] - ETA: 0s - loss: 9.8240e-05 - accuracy: 1.00 - 0s 340us/step - loss: 0.0093 - accuracy: 1.0000
Epoch 165/200
51/51 [==============================] - 0s 406us/step - loss: 0.0213 - accuracy: 1.0000
Epoch 166/200
51/51 [==============================] - 0s 414us/step - loss: 0.0127 - accuracy: 1.0000
Epoch 167/200
51/51 [==============================] - 0s 290us/step - loss: 0.0030 - accuracy: 1.0000
Epoch 168/200
51/51 [==============================] - 0s 291us/step - loss: 0.0067 - accuracy: 1.0000
Epoch 169/200
51/51 [====================

In [12]:
# save model to file
pickle.dump(model, open("./pickle/robi_en_model.pkl", "wb"))

# save all of our data structures
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "./pickle/robi_en_data.pkl", "wb" ) )